<a href="https://colab.research.google.com/github/Viliuskerutis/AML/blob/main/AML_MP1_Kerutis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Which country has the most deaths per million people in June?

In [15]:
import pandas as pd
import numpy as np

In [27]:
#df = pd.read_csv(r'C:/Users/kerut/Desktop/KTU/Magistras/Pirmas Kursas/Advanced Machine Learning/owid-covid-data.csv', index_col=0)
df = pd.read_csv('https://raw.githubusercontent.com/Viliuskerutis/AML/main/owid-covid-data.csv', low_memory=False)
print (df)

      iso_code continent  ... life_expectancy human_development_index
0          AFG      Asia  ...           64.83                   0.498
1          AFG      Asia  ...           64.83                   0.498
2          AFG      Asia  ...           64.83                   0.498
3          AFG      Asia  ...           64.83                   0.498
4          AFG      Asia  ...           64.83                   0.498
...        ...       ...  ...             ...                     ...
49432      NaN       NaN  ...             NaN                     NaN
49433      NaN       NaN  ...             NaN                     NaN
49434      NaN       NaN  ...             NaN                     NaN
49435      NaN       NaN  ...             NaN                     NaN
49436      NaN       NaN  ...             NaN                     NaN

[49437 rows x 41 columns]
